In [1]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [55]:
# Set the executable path and initialize Splinter
executable_path = {'executable_path': ChromeDriverManager(version = '98.0.4758.102').install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\M037228\.wdm\drivers\chromedriver\win32\98.0.4758.102]


### Visit the NASA Mars News Site

In [3]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
news_soup = bs(html, 'html.parser')

slide_elem = news_soup.select_one('div.list_text')

In [5]:
slide_elem.find('div', class_='content_title')

<div class="content_title">Sensors on Mars 2020 Spacecraft Answer Long-Distance Call From Earth</div>

In [6]:
# Use the parent element to find the first a tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

'Sensors on Mars 2020 Spacecraft Answer Long-Distance Call From Earth'

In [7]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"Instruments tailored to collect data during the descent of NASA's next rover through the Red Planet's atmosphere have been checked in flight."

### JPL Space Images Featured Image

In [8]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [9]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [10]:
# Parse the resulting html with soup
html = browser.html
img_soup = bs(html, 'html.parser')
img_soup

<html class="fancybox-margin fancybox-lock"><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
<!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<title>Space Image</title>
<style type="text/css">.fancybox-margin{margin-right:17px;}</style></head>
<body>
<div class="header">
<nav class="navbar navbar-expand-lg">
<a class="navbar-brand" href="#"><img id="logo" src="image/nasa.png"/><span class="logo">Jet Propulsion Laboratory</span>
<span class="logo1">California Institute of Technology</span></a>
<button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-target="#navbarNav" data-t

In [11]:
# find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars3.jpg'

In [12]:
# Use the base url to create an absolute url
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

### Mars Facts

In [13]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.head()

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"


In [14]:
df.columns=['Description', 'Mars', 'Earth']
df.set_index('Description', inplace=True)
df

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [15]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [108]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)

In [32]:
# # 2. Create a list to hold the images and titles.
hemisphere_image_urls = []
hemispheres = {}
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')
# Retrieve all elements that contain book information
#articles = soup.find_all('article', class_='product_pod')
articles = soup.find_all('div', class_='item')
url_list = []
title_list = []
    # Iterate through each book
for article in articles:
# Use Beautiful Soup's find() method to navigate and retrieve attributes
    #find title
    h3 = article.find('h3')
    title = h3.text.strip()
    title_list.append(title)
    #find link
    link = article.find('img')['src']
    url_list.append(link)

    hemispheres = {}

image_urls = ['https://marshemispheres.com/' + url for url in url_list]


print(image_urls)


    

In [81]:
# # 2. Create a list to hold the images and titles.
hemisphere_image_urls = []
hemispheres = {}
url_root = 'https://marshemispheres.com/'
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')
# Retrieve all elements that contain book information
#articles = soup.find_all('article', class_='product_pod')
articles = soup.find_all('div', class_='item')
url_list = []
title_list = []
    # Iterate through each book
for article in articles:
    h3 = article.find('h3')
    title = h3.text.strip()







image_urls = ['https://marshemispheres.com/' + url for url in url_list]

print(image_urls)

{}


In [246]:

# cerberus = 'https://marshemispheres.com/cerberus.html'

# browser.visit(cerberus)

html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

# browser.find_by_css('h1')
# browser.find_by_xpath('//h1')
# browser.find_by_tag('h1')
# browser.find_by_name('name')
# browser.find_by_text('Hello World!')
# browser.find_by_id('firstheader')
# browser.find_by_value('query')
#z = browser.find_by_value('image').first
#first_found = browser.find_by_name('').first
# Print all ten headlines

#results = soup.find_all('li')
#esults = browser.find_by_tag('li').first
#results = soup.find_by_tag('h1')

#for result in soup.find_all('a'):
    # Error handling

        # Identify and return title of listing
    #title = result.find('a', target_="blank").text
        # Identify and return price of listing
    #price = result.a.span.text
        # Identify and return link to listing
    #link = result.a['href']
    # print(link.get('href'))

    #     # Print results only if title, price, and link are available
    
    # print('-------------')
    #print(title)
    #print(price)
    #print(link)
#soup.title
#soup.find_all('h2')
links = []
for link in soup.find_all('a'):
    links.append(link.get('href'))

image_link = links[3]
print(image_link)


images/full.jpg


In [191]:
browser.visit(url)
# # 2. Create a list to hold the images and titles.
hemisphere_image_urls = []
hemispheres = {}
url_root = 'https://marshemispheres.com/'
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')
# Retrieve all elements that contain book information
#articles = soup.find_all('article', class_='product_pod')
articles = soup.find_all('div', class_='item')
url_list = []
title_list = []
    # Iterate through each book
for article in articles:
# Use Beautiful Soup's find() method to navigate and retrieve attributes
    #find title
    #find link
    # link = article.find('a')['href']
    # url_list.append(link)
    # for url in url_list:
    # link.click()
    h3 = article.find('h3')
    title = h3.text.strip()
    #print(title)
    browser.links.find_by_partial_text(title).click()
    print(soup.find_all('h2'))
    #find_link.click()
    #image_link = soup.find('h3')
    #divs = browser.find_by_tag("li")
    
    #image_link = soup.find_all('li', target_='_blank')
    #category = image_link.find_all()
    #print(urls)
    #image_link = browser.find('a')['href']
    #print(image_link)
    #browser.back()
    # image_link = .find('a')['href']
    # h2 = article.find('h2')
    # title = h2.text.strip()
    # print(title)
    # print(image_link)    


#print(hemispheres)

image_urls = ['https://marshemispheres.com/' + url for url in url_list]

[<h2>Products</h2>]


ElementDoesNotExist: no elements could be found with link by partial text "Schiaparelli Hemisphere Enhanced"

In [256]:
browser.visit(url_root)
# # 2. Create a list to hold the images and titles.
hemisphere_image_urls = []
hemispheres = {}
url_root = 'https://marshemispheres.com/'

html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')
# Retrieve all elements that contain book information

articles = soup.find_all('div', class_='item')
url_list = []
title_list = []

for article in articles:
    h3 = article.find('h3')
    title = h3.text.strip()
    # print(title)
    title_list.append(title)
    browser.links.find_by_partial_text(title).click()
    x = browser.find_by_tag('h2')
    page_title = x.text.strip()
    # print(page_title)
    links = []
    for link in soup.find_all('a'):
        #links.append(soup.get('href'))
        s = soup.find_all('src')
        print(s)
        #print(link.find_all['j'])


    



        # links.append(target)
        # links.append(link.get('href'))
        # print(links)
    browser.back()
#url_list = links[3]
# print(links)


[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [ ]:
for article in articles:
    h3 = article.find('h3')
    title = h3.text.strip()
    # print(title)
    title_list.append(title)
    browser.links.find_by_partial_text(title).click()
    x = browser.find_by_tag('h2')
    page_title = x.text.strip()
    # print(page_title)
    links = []
    for link in soup.find_all('li'):
        if link.has_attr('target'):
            print(link)

In [229]:
url_list = links[3:11]
urls = []
for url in url_list:
    url = f'https://marshemispheres.com/{url}'
    if url not in urls:
        urls.append(url)
url_list

title_list
print(urls)

['https://marshemispheres.com/cerberus.html', 'https://marshemispheres.com/schiaparelli.html', 'https://marshemispheres.com/syrtis.html', 'https://marshemispheres.com/valles.html']


[<h2>Products</h2>]


In [230]:
first = {}
second = {}
third = {}
fourth = {}
first['img_url'] = urls[0]
second['img_url'] = urls[1]
third['img_url'] = urls[2]
fourth['img_url'] = urls[3]

first['title'] = title_list[0]
second['title'] = title_list[1]
third['title'] = title_list[2]
fourth['title'] = title_list[3]


print(first)
print(second)
print(third)
print(fourth)

# hemispheres = [[ i for i, j in hemisphere_image_urls ], [ j for i, j in title_list ]]

#hemispheres

{'img_url': 'https://marshemispheres.com/cerberus.html', 'title': 'Cerberus Hemisphere Enhanced'}
{'img_url': 'https://marshemispheres.com/schiaparelli.html', 'title': 'Schiaparelli Hemisphere Enhanced'}
{'img_url': 'https://marshemispheres.com/syrtis.html', 'title': 'Syrtis Major Hemisphere Enhanced'}
{'img_url': 'https://marshemispheres.com/valles.html', 'title': 'Valles Marineris Hemisphere Enhanced'}


In [35]:
hemisphere_image_urls = [first,second,third,fourth]
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [18]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [19]:
# 5. Quit the browser
browser.quit()